In [1]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv

# Azure AI Projects
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from azure.core.exceptions import HttpResponseError
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    SharepointTool,
    FabricTool,
    ToolSet,
)

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

# helper functions
from utils.utilityfucntions import print_agent_summary

# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.


In [2]:
# The connection string format should be: <HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>
conn_str = os.environ["AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"]

# Create the AIProjectClient using the connection string and explicit credential
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=conn_str,
)

### **1. Creating an Azure AI Agent with SharePoint Integration (Tool)**

SharePoint as a data source allows you to ground your Azure AI Agents with documents stored in SharePoint securely. You can connect to your SharePoint site, such as `contoso.sharepoint.com/sites/policies`. When a user sends a query, Azure AI Agents will determine if SharePoint should be leveraged or not. If so, it will send the query via the SharePoint tool, which checks if the user has an M365 Copilot license and uses the end user’s identity to retrieve relevant documents they have access to. The scope of retrieval includes all supported documents in this SharePoint site. Lastly, Azure AI Agents will generate responses based on the retrieved information.

With SharePoint integration, we will support **OBO (On-Behalf-Of) authentication**, which allows the SharePoint tool to retrieve relevant documents based on the end user’s identity and access.

**Prerequisites**
- Existing SharePoint site, and ensure developers have access to this SharePoint site.
- Developers and end users must have an M365 Copilot license.
- Ensure your AOAI resource and AI project are in one of the following regions: `westus`, `japaneast`, `francecentral`.

+ **RBAC Roles**
    - To CRUD a SharePoint tool in Azure AI Agent, ensure you have the **AI Developer** role.
    - Ensure end users have the **AI Developer** role to enable OBO authentication.

> Please visit [how-to\setup-sharepoint-azure-ai-agentmd](how-to\setup-sharepoint-azure-ai-agent.md) to lear how-to set Up Tools SharepointTool.

**Supported Capabilities**
- Grounding with all supported documents in a SharePoint site.
- Supported document types: PDF, Word, PPT, TXT, .aspx (text data only).
- Automatic indexing of updated documents.
- Responses based on the end user’s document access.
- Connection to a maximum of 1 SharePoint site.

**Known Limitations**
- Retrieval and grounding quality issues, especially with complex formatting, tables, columns, and images.
- Text-sparse file types (e.g., PPT) may return poorer results than text-rich types (e.g., DOCX).
- Grounding with specific libraries or multiple sites is not supported.

In [3]:
# 1. Set up the model name from environment variable
deployment_name = os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME")
if not deployment_name:
    logger.error("Environment variable 'AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME' is not set.")
    exit(1)

# 2. Initialize the AI Project Client (assumed to be done in a previous cell)
# 2a. Initialize the SharePoint tool using the connection ID.
# For connection setup details, please refer to `how-to/setup-tools-in-ai-foundry.md`
try:
    sharepoint_connection = project_client.connections.get(
        connection_name=os.environ["TOOL_CONNECTION_NAME_SHAREPOINT"]
    )
except KeyError:
    logger.error("Environment variable 'TOOL_CONNECTION_NAME_SHAREPOINT' is not set.")
    exit(1)

conn_id = sharepoint_connection.id
logger.info(f"SharePoint Connection ID: {conn_id}")

# Create a SharePointTool instance with the connection ID
sharepoint = SharepointTool(connection_id=conn_id)

# 3. Create a SharePoint-powered AI Agent
try:
    sharepoint_agent = project_client.agents.create_agent(
        model=deployment_name,
        name="my-sharepoint-assistant",
        description=(
            "A SharePoint-integrated AI assistant designed to search, retrieve, "
            "and summarize documents stored in SharePoint. This assistant helps users "
            "quickly find relevant information while ensuring responses include "
            "document references for validation."
        ),
        instructions=(
            "You are an AI-powered assistant specialized in searching and retrieving "
            "documents from SharePoint. Your primary role is to find the most relevant "
            "documents based on user queries and provide clear, concise summaries.\n\n"
            "**Response Guidelines:**\n"
            "1. Always retrieve the most relevant document(s) from SharePoint.\n"
            "2. Provide a summary of the key information from the retrieved document(s).\n"
            "3. Every response **must include the link** to the original document for reference.\n"
            "4. If no relevant document is found, respond with: 'No matching documents found in SharePoint.'\n"
            "5. Use professional, structured language, and avoid speculation.\n\n"
            "**Example Responses:**\n"
            "✅ 'The requested policy document is available here: [Document Link]. Based on its content, "
            "the key points are...'\n"
            "❌ 'I think this might be what you're looking for...' (Avoid speculation.)"
        ),
        tools=sharepoint.definitions,
        headers={"x-ms-enable-preview": "true"},
        temperature=1,
        top_p=1,
        metadata={
            "use_case": "Enterprise SharePoint Search",
            "data_source": "SharePoint",
            "response_validation": "Must include source link"
        },
    )

    logger.info(f"Created Agent ID: {sharepoint_agent.id}")
    logger.info(f"Agent Metadata: {sharepoint_agent.metadata}")

except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"Error Message: {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"Non-JSON Error Content: {e.response.content}")
    exit(1)

print_agent_summary(sharepoint_agent)


2025-03-19 20:05:58,594 - micro - MainProcess - INFO     SharePoint Connection ID: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/ContosoAgentDemoSharepoint (1095327310.py:<module>:19)
INFO:micro:SharePoint Connection ID: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/ContosoAgentDemoSharepoint
2025-03-19 20:06:01,606 - micro - MainProcess - INFO     Created Agent ID: asst_a07FA4kFCKoTEtEpn2RETeqX (1095327310.py:<module>:61)
INFO:micro:Created Agent ID: asst_a07FA4kFCKoTEtEpn2RETeqX
2025-03-19 20:06:01,611 - micro - MainProcess - INFO     Agent Metadata: {'use_case': 'Enterprise SharePoint Search', 'data_source': 'SharePoint', 'response_validation': 'Must include source link'} (1095327310.py:<module>:62)
INFO:micro:Agent Metadata: {'use_case': 'Enterprise 


=== Agent Creation Summary ===
Agent Name    : my-sharepoint-assistant
Agent ID      : asst_a07FA4kFCKoTEtEpn2RETeqX
Agent Metadata:
  - use_case: Enterprise SharePoint Search
  - data_source: SharePoint
  - response_validation: Must include source link


#### **Trying to Find Information About Dexcom G7 CGM (Info Available in SharePoint)**

  Our agent ha been configured with the necessary permissions and equipped with tools like `SharepointTool` to search for the relevant document, validate its existence, and parse its contents. Once the document is retrieved, the agent will extract critical features such as real-time glucose monitoring, compact design, improved accuracy, integration with smart devices, and customizable alerts, ensuring the information is accurate and formatted for digital use. This process requires robust error handling to manage scenarios where the document is missing or inaccessible, and the output should be structured for easy integration into other systems or reports.

In [4]:
try:
    # 1. Create a thread for the conversation
    thread = project_client.agents.create_thread()
    logger.info(f"Created Thread ID: {thread.id}")

    # 2. Create a user message on that thread
    user_message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What are the key features from Dexcom G7 CGM System?"
    )
    logger.info(f"Created User Message ID: {user_message.id}")

    # 3. Create a run to process the conversation
    run = project_client.agents.create_run(
        thread_id=thread.id, agent_id=sharepoint_agent.id
    )
    logger.info("Run created. Polling for status...")

    # 4. Poll until run completes or fails
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(2)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        logger.info(f"Current run ID: {run.id}")
        logger.info(f"Current run status: {run.status}")

    logger.info(f"Run finished with status: {run.status}")

    # 5. Retrieve and display the conversation history (oldest to newest)
    conversation_history = project_client.agents.list_messages(thread_id=thread.id)
    logger.info("----- Conversation History -----")
    for msg in conversation_history.data:
        if msg.content and isinstance(msg.content[-1], MessageTextContent):
            logger.info(f"{msg.role.upper()}: {msg.content[-1].text.value}")
except Exception as e:
    logger.error(f"An error occurred during agent processing: {e}")

2025-03-19 20:06:02,106 - micro - MainProcess - INFO     Created Thread ID: thread_waaxnu9VuHW7gvTjp783zLfb (127510436.py:<module>:4)
INFO:micro:Created Thread ID: thread_waaxnu9VuHW7gvTjp783zLfb
2025-03-19 20:06:02,585 - micro - MainProcess - INFO     Created User Message ID: msg_OMrxtcCvbP00neUqI2ZNRIka (127510436.py:<module>:12)
INFO:micro:Created User Message ID: msg_OMrxtcCvbP00neUqI2ZNRIka
2025-03-19 20:06:04,918 - micro - MainProcess - INFO     Run created. Polling for status... (127510436.py:<module>:18)
INFO:micro:Run created. Polling for status...
2025-03-19 20:06:07,337 - micro - MainProcess - INFO     Current run ID: run_Q91mAoNtPrkDO5CTnRJUd3DW (127510436.py:<module>:24)
INFO:micro:Current run ID: run_Q91mAoNtPrkDO5CTnRJUd3DW
2025-03-19 20:06:07,341 - micro - MainProcess - INFO     Current run status: RunStatus.IN_PROGRESS (127510436.py:<module>:25)
INFO:micro:Current run status: RunStatus.IN_PROGRESS
2025-03-19 20:06:09,769 - micro - MainProcess - INFO     Current run ID:

Great News! Our SharePoint Knowledge Retrieval Agent Successfully Found the Necessary Information!

### **Creating an Azure AI Agent with Fabric Integration (Tool)**

Integrate your Azure AI Agent with Fabric `Data agent` to unlock powerful data analysis capabilities. Fabric `Data Agent` transforms enterprise data into conversational Q&A systems, allowing users to interact with data through chat and uncover actionable insights effortlessly.

When a user sends a query, the Azure AI Agent first determines if Fabric `Data agent` should be leveraged. If so, it uses the end user’s identity to generate queries over accessible data, and then returns responses based on the queried results. With **on-behalf-of (OBO) authorization**, this integration simplifies secure access to enterprise data in Fabric while ensuring robust protection and proper access control.

**Prerequisites**
- Published Fabric AI Skill: A published Fabric AI Skill is required (or use the provided Fabric link to access pre-created skills).
- Permission/Role Assignment:
    + Access to AI Skill: Users must have at least “Read” access to the AI Skill and connected data sources.
    + RBAC of Foundry Project: End users need the AI Developer role.

+ **RBAC Roles**
    - To CRUD a SharePoint tool in Azure AI Agent, ensure you have the **AI Developer** role.
    - Ensure end users have the **AI Developer** role to enable OBO authentication.

> For detailed step-by-step instructions, please visit [How to Set Up Fabric Azure AI Agent](how-to\setup-fabric-azure-ai-agent.md).

**Known Limitations**
- Please review the [Data Agent documentation](https://learn.microsoft.com/en-us/fabric/data-science/concept-ai-skill) for additional details and limitations.

In [ ]:

conn_id = project_client.connections.get(
    connection_name=os.environ["TOOL_CONNECTION_NAME_FABRIC"],
)

# 4. Initialize Fabric Tool
fabric_tool = FabricTool(connection_id=conn_id.id)

# 5. Create the Fabric AI Agent
try:
    fabric_agent = project_client.agents.create_agent(
        model=deployment_name,
        name="fabric-intelligence-agent",
        description=(
            "An AI-powered assistant designed to interact with Microsoft Fabric, "
            "fetch relevant data, analyze insights, and ensure structured responses "
            "with validated references. The agent helps users extract valuable "
            "business intelligence from Fabric data sources."
        ),
        instructions=(
            "You are an AI assistant specialized in retrieving and analyzing data "
            "from Microsoft Fabric. Your primary goal is to help users extract insights "
            "by querying structured and unstructured data sources within Fabric. \n\n"
            "**Response Guidelines:**\n"
            "1. Always retrieve the most relevant dataset(s) from Fabric.\n"
            "2. Provide a structured summary of key findings, ensuring accuracy.\n"
            "3. Every response **must include a link** to the original data source for verification.\n"
            "4. If no relevant data is found, respond with: 'No relevant data found in Fabric.'\n"
            "5. Use precise, professional, and structured language; avoid speculation.\n\n"
            "**Example Responses:**\n"
            "✅ *'The requested sales performance data is available here: [Dataset Link]. Key insights: ...'* \n"
            "❌ *'I think this might be useful...'* (Avoid vague responses.)"
        ),
        tools=fabric_tool.definitions,
        headers={"x-ms-enable-preview": "true"},
        temperature=0.7,
        top_p=1,
        metadata={
            "use_case": "Microsoft Fabric Data Analysis",
            "data_source": "Fabric",
            "response_validation": "Must include source link"
        },
    )

    logger.info(f"Created Agent ID: {fabric_agent.id}")
    logger.info(f"Agent Metadata: {fabric_agent.metadata}")

except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"Error Message: {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"Non-JSON Error Content: {e.response.content}")

print_agent_summary(fabric_agent)

2025-03-19 20:06:13,485 - micro - MainProcess - INFO     Created Agent ID: asst_28CfsW8cEBVvG3HxNSporJMr (4065364071.py:<module>:44)
INFO:micro:Created Agent ID: asst_28CfsW8cEBVvG3HxNSporJMr
2025-03-19 20:06:13,490 - micro - MainProcess - INFO     Agent Metadata: {'use_case': 'Microsoft Fabric Data Analysis', 'data_source': 'Fabric', 'response_validation': 'Must include source link'} (4065364071.py:<module>:45)
INFO:micro:Agent Metadata: {'use_case': 'Microsoft Fabric Data Analysis', 'data_source': 'Fabric', 'response_validation': 'Must include source link'}



=== Agent Creation Summary ===
Agent Name    : fabric-intelligence-agent
Agent ID      : asst_28CfsW8cEBVvG3HxNSporJMr
Agent Metadata:
  - use_case: Microsoft Fabric Data Analysis
  - data_source: Fabric
  - response_validation: Must include source link


In [6]:
try:
    # 1. Create a thread for the conversation
    thread = project_client.agents.create_thread()
    logger.info(f"Created Thread ID: {thread.id}")

    # 2. Create a user message on that thread
    user_message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="How does Product A compare to Product B in terms of MARD percentage across different glucose ranges? Use the tools to analyze the data and provide a summary.",
    )
    logger.info(f"Created User Message ID: {user_message.id}")

    # 3. Create a run to process the conversation
    run = project_client.agents.create_run(
        thread_id=thread.id, agent_id=fabric_agent.id
    )
    logger.info("Run created. Polling for status...")

    # 4. Poll until run completes or fails
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(2)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        logger.info(f"Current run ID: {run.id}")
        logger.info(f"Current run status: {run.status}")

    logger.info(f"Run finished with status: {run.status}")

    # 5. Retrieve and display the conversation history (oldest to newest)
    conversation_history = project_client.agents.list_messages(thread_id=thread.id)
    logger.info("----- Conversation History -----")
    for msg in conversation_history.data:
        if msg.content and isinstance(msg.content[-1], MessageTextContent):
            logger.info(f"{msg.role.upper()}: {msg.content[-1].text.value}")
except Exception as e:
    logger.error(f"An error occurred during agent processing: {e}")

2025-03-19 20:06:13,916 - micro - MainProcess - INFO     Created Thread ID: thread_2o5rdFBPTml7sG9gL0U9jksc (1545354897.py:<module>:4)
INFO:micro:Created Thread ID: thread_2o5rdFBPTml7sG9gL0U9jksc
2025-03-19 20:06:14,398 - micro - MainProcess - INFO     Created User Message ID: msg_pNdwFp2tdfZ5zuiAjOvNn0qk (1545354897.py:<module>:12)
INFO:micro:Created User Message ID: msg_pNdwFp2tdfZ5zuiAjOvNn0qk
2025-03-19 20:06:15,920 - micro - MainProcess - INFO     Run created. Polling for status... (1545354897.py:<module>:18)
INFO:micro:Run created. Polling for status...
2025-03-19 20:06:18,325 - micro - MainProcess - INFO     Current run ID: run_6dLjWCxALCMIxCsdWqIipf83 (1545354897.py:<module>:24)
INFO:micro:Current run ID: run_6dLjWCxALCMIxCsdWqIipf83
2025-03-19 20:06:18,329 - micro - MainProcess - INFO     Current run status: RunStatus.FAILED (1545354897.py:<module>:25)
INFO:micro:Current run status: RunStatus.FAILED
2025-03-19 20:06:18,333 - micro - MainProcess - INFO     Run finished with st

### **Creating an Azure AI Agent with Bing Integration (Tool) + Azure AI Search**

Integrate your Azure AI Agent with Azure AI Search and Bing Search to unlock powerful enterprise and real-time public web data capabilities. By combining the strengths of Azure AI Search for internal data with Bing Search for live web content, your agent can deliver enriched, context-aware responses.

When a user sends a query (for example, "Should I take an umbrella with me today? I'm in Seattle."), the Azure AI Agent determines whether to leverage Azure AI Search for enterprise data, Bing Search for public data, or both. It then retrieves the necessary data—applying secure, role-based access—and generates a comprehensive, human-readable response that includes required citations and links.

**Prerequisites**
+ Published Azure AI Search Index: Ensure you have a properly configured and published Azure AI  Search index containing your data.
+ Grounding with Bing Search Resource: Create a Bing Search resource to enable real-time public web data retrieval.

> For detailed step-by-step instructions, please visit [How to Set Up Bing Tool](how-to\fabric.md) and [How to Set Up Azure AI Search Tool](how-to\setup-fabric-azure-ai-agent.md).

**Known Limitations**
- Please review the [documentation](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview) for additional details and limitations.


In [ ]:
bing_conn_id = project_client.connections.get(
    connection_name=os.environ["TOOL_CONNECTION_NAME_BING"],
)
bing_tool = BingGroundingTool(connection_id=bing_conn_id.id)

In [8]:
search_conn_id = project_client.connections.get(
    connection_name=os.environ["TOOL_CONNECTION_NAME_SEARCH"],
)
azure_ai_search_tool = AzureAISearchTool(index_connection_id=search_conn_id.id,
                                         index_name="ai-agentic-index")

Combine All Tools into a ToolSet
This step creates a custom ToolSet that includes all the tools configured earlier. It also adds a LoggingToolSet subclass to log the inputs and outputs of function calls.

In [9]:
toolset = ToolSet()
toolset.add(bing_tool)
toolset.add(azure_ai_search_tool)

In [10]:
# Set up the model name from environment variable
deployment_name = os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME")
if not deployment_name:
    logger.error("Environment variable 'AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME' is not set.")
    exit(1)

# Create the Reflective Agent with Dual Knowledge Access
try:
    reflective_agent = project_client.agents.create_agent(
        model=deployment_name,
        name="reflective-dual-agent",
        description=(
            "A reflective AI-powered assistant designed to integrate both internal enterprise data "
            "via Azure AI Search and real-time public data via Bing Search. This agent ensures that "
            "every response includes citations from internal and/or external sources, providing a comprehensive "
            "and verified answer to each query."
        ),
        instructions=(
            "You are a reflective AI assistant equipped with two distinct tools: one for accessing internal enterprise data "
            "(Azure AI Search) and another for retrieving real-time public information (Bing Search). Your goal is to "
            "provide accurate and structured responses by combining data from both sources as needed. \n\n"
            "**Response Guidelines:**\n"
            "1. Retrieve the most relevant information from internal enterprise data using Azure AI Search.\n"
            "2. Augment your answers with real-time public data from Bing Search when applicable.\n"
            "3. Every response **must include a citation** linking to the original source(s) from internal and/or external data.\n"
            "4. If no relevant data is found, respond with: 'No relevant data found.'\n"
            "5. Use precise, professional language and ensure that both internal and external data are appropriately referenced.\n\n"
            "**Example Responses:**\n"
            "✅ *'The requested sales performance data is available here: [Internal Dataset Link], and the latest market trends can be found here: [Bing Search Result Link].'* \n"
            "❌ *'I think this might be useful...'* (Avoid vague or unverified responses.)"
        ),
        toolset=toolset,
        headers={"x-ms-enable-preview": "true"},
        temperature=0.7,
        top_p=1,
        metadata={
            "use_case": "Dual Knowledge Integration for Enterprise and Real-Time Data",
            "data_source": "Internal (Azure AI Search) and External (Bing Search)",
            "response_validation": "Must include citation(s) from internal and/or external sources"
        },
    )

    logger.info(f"Created Agent ID: {reflective_agent.id}")
    logger.info(f"Agent Metadata: {reflective_agent.metadata}")

except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"Error Message: {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"Non-JSON Error Content: {e.response.content}")
    exit(1)

print_agent_summary(reflective_agent)


2025-03-19 20:06:19,760 - micro - MainProcess - INFO     Created Agent ID: asst_7s7ZAeePGZpYDCo2xFR5Htsp (460002155.py:<module>:43)
INFO:micro:Created Agent ID: asst_7s7ZAeePGZpYDCo2xFR5Htsp
2025-03-19 20:06:19,764 - micro - MainProcess - INFO     Agent Metadata: {'use_case': 'Dual Knowledge Integration for Enterprise and Real-Time Data', 'data_source': 'Internal (Azure AI Search) and External (Bing Search)', 'response_validation': 'Must include citation(s) from internal and/or external sources'} (460002155.py:<module>:44)
INFO:micro:Agent Metadata: {'use_case': 'Dual Knowledge Integration for Enterprise and Real-Time Data', 'data_source': 'Internal (Azure AI Search) and External (Bing Search)', 'response_validation': 'Must include citation(s) from internal and/or external sources'}



=== Agent Creation Summary ===
Agent Name    : reflective-dual-agent
Agent ID      : asst_7s7ZAeePGZpYDCo2xFR5Htsp
Agent Metadata:
  - use_case: Dual Knowledge Integration for Enterprise and Real-Time Data
  - data_source: Internal (Azure AI Search) and External (Bing Search)
  - response_validation: Must include citation(s) from internal and/or external sources


In [13]:
try:
    # 1. Create a thread for the conversation
    thread = project_client.agents.create_thread()
    logger.info(f"Created Thread ID: {thread.id}")

    # 2. Create a user message on that thread
    user_message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            "Can you provide the system architecture of Product A? Additionally,"
            "Please use real-time data and search for similar stores close to 60601 "
            "that could sell continuous glucose monitoring (iCGM) devices."
        ),
    )
    logger.info(f"Created User Message ID: {user_message.id}")

    # 3. Create a run to process the conversation
    run = project_client.agents.create_run(
        thread_id=thread.id, agent_id=reflective_agent.id
    )
    logger.info("Run created. Polling for status...")

    # 4. Poll until run completes or fails
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(2)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        logger.info(f"Current run ID: {run.id}")
        logger.info(f"Current run status: {run.status}")

    logger.info(f"Run finished with status: {run.status}")

    # 5. Retrieve and display the conversation history (oldest to newest)
    conversation_history = project_client.agents.list_messages(thread_id=thread.id)
    logger.info("----- Conversation History -----")
    for msg in conversation_history.data:
        if msg.content and isinstance(msg.content[-1], MessageTextContent):
            logger.info(f"{msg.role.upper()}: {msg.content[-1].text.value}")
except Exception as e:
    logger.error(f"An error occurred during agent processing: {e}")

2025-03-19 20:10:34,806 - micro - MainProcess - INFO     Created Thread ID: thread_tC7lcJD28C6m1w9J89nqsW3M (2377848834.py:<module>:4)
INFO:micro:Created Thread ID: thread_tC7lcJD28C6m1w9J89nqsW3M
2025-03-19 20:10:35,229 - micro - MainProcess - INFO     Created User Message ID: msg_IYxGLl6Mt4x7UyCeV2nT7qgX (2377848834.py:<module>:16)
INFO:micro:Created User Message ID: msg_IYxGLl6Mt4x7UyCeV2nT7qgX
2025-03-19 20:10:36,714 - micro - MainProcess - INFO     Run created. Polling for status... (2377848834.py:<module>:22)
INFO:micro:Run created. Polling for status...
2025-03-19 20:10:39,085 - micro - MainProcess - INFO     Current run ID: run_xIM4gpc3f4HfPaV7mbtqa88v (2377848834.py:<module>:28)
INFO:micro:Current run ID: run_xIM4gpc3f4HfPaV7mbtqa88v
2025-03-19 20:10:39,090 - micro - MainProcess - INFO     Current run status: RunStatus.IN_PROGRESS (2377848834.py:<module>:29)
INFO:micro:Current run status: RunStatus.IN_PROGRESS
2025-03-19 20:10:41,585 - micro - MainProcess - INFO     Current ru